In [1]:
!pip install -U langchain-community langchain-text-splitters chromadb

In [3]:
pip install ipywidgets

   ---------------------------------------- 0.0/914.9 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/914.9 kB ? eta -:--:--
   ---------------------- ----------------- 524.3/914.9 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 914.9/914.9 kB 1.6 MB/s  0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.2 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.2 MB 1.9 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.2 MB 1.8 MB/s eta 0:00:01
   ----------------------- ---------------- 1.3/2.2 MB 1.8 MB/s eta 0:00:01
   --------------------------------- ------ 1.8/2.2 MB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 1.7 MB/s  0:00:01

   ---------------------------------------- 0/3 [widgetsnbextension]
   ------------- -------------------------- 1/3 [jupyterlab_widgets]
   -------------------------- ------------- 2

In [1]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [3]:
pip install -U langchain-ollama


   -------------------- ------------------- 1/2 [langchain-ollama]
   ---------------------------------------- 2/2 [langchain-ollama]

Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain_ollama import OllamaLLM, OllamaEmbeddings

llm = OllamaLLM(model="smollm2:360m")   # or llama3.2:1b

embeddings = OllamaEmbeddings(
    model="nomic-embed-text:latest"
)

In [4]:
os.path.exists(r"C:\Users\Dell\OneDrive\Desktop\college\GenAI2026\data.txt")

True

In [11]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = TextLoader(
    r"C:\Users\Dell\OneDrive\Desktop\college\GenAI2026\data.txt",
    encoding="utf-8"
)

docs = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

splits = splitter.split_documents(docs)

In [14]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = OllamaEmbeddings(
    model="nomic-embed-text:latest"
)

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory="chroma_db"
)

vectorstore.persist()

In [17]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="smollm2:360m")

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_template("""
Answer the question using only the context.

Context:
{context}

Question:
{question}
""")

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": lambda x: x
    }
    | prompt
    | llm
    | StrOutputParser()
)

response = rag_chain.invoke("What is AI?")
print(response)

Answer: AI, or Artificial Intelligence, refers to the simulation of human intelligence in machines that are programmed and capable of performing tasks requiring human-like reasoning, problem-solving, perception, and learning.


In [2]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

# Create embeddings
embedding = OllamaEmbeddings(model="nomic-embed-text")

# Create vector store (example with dummy texts)
texts = ["This is a sample document", "LangChain is useful"]

vectorstore = Chroma.from_texts(texts, embedding=embedding)

C:\Users\Dell\AppData\Local\Temp\ipykernel_21636\3125547128.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(model="nomic-embed-text")


In [3]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = OllamaLLM(model="smollm2:360m")

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 6}  # fetch more chunks for packing
)

In [4]:
rephrase_prompt = ChatPromptTemplate.from_template("""
Rewrite the user question to make it more detailed and retrieval-friendly.

Original Question:
{question}

Rewritten Question:
""")

rephrase_chain = rephrase_prompt | llm | StrOutputParser()

In [5]:
def pack_context(docs, max_chars=2000):
    packed = ""
    for doc in docs:
        if len(packed) + len(doc.page_content) > max_chars:
            break
        packed += doc.page_content + "\n\n"
    return packed

In [7]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = OllamaLLM(model="smollm2:360m")

# Prompt
prompt = ChatPromptTemplate.from_template("""
Answer the question based only on the context below:

{context}

Question: {question}
""")

# Format docs
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# RAG chain
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": lambda x: x
    }
    | prompt
    | llm
    | StrOutputParser()
)

# ✅ Define function
def ask_rag(question):
    return rag_chain.invoke(question)

In [9]:
response = ask_rag("What is this document about?")
print(response)

This document is for someone looking to learn more about LangChain. It might contain information about its benefits, features, and what it can do in various industries.
